<a href="https://colab.research.google.com/github/snbigft/update_ambos_matrix_from_REST/blob/main/update_ambos_matrix_from_REST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install fastapi uvicorn psycopg2-binary nest-asyncio pyngrok python-dotenv requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.2 MB/s eta 0:00:00


In [4]:
import requests
import psycopg2
from psycopg2 import sql

# Configuración de la base de datos
DB_HOST = "ep-snowy-dawn-a8omlpke-pooler.eastus2.azure.neon.tech"
DB_NAME = "neondb"
DB_USER = "neondb_owner"
DB_PASSWORD = "npg_BcpnId62FYhS"

# URL de la API
LOTTO_API_URL = "https://www.lotto-italia.it/gdl/estrazioni-e-vincite/estrazioni-del-lotto.json"

# Función para conectar a la base de datos
def get_db_connection():
    return psycopg2.connect(
        host=DB_HOST,
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        sslmode="require"
    )

# Pedir la fecha como input
fecha = input("Introduce la fecha en formato YYYYMMDD: ")

# Obtener datos de la API
response = requests.post(LOTTO_API_URL, json={"data": fecha})
data = response.json()

if data.get("esito") != "OK":
    print("Error en la respuesta de la API.")
    exit()

# Procesar los datos
conn = get_db_connection()
cursor = conn.cursor()
commit_chunk = 10000
update_counter = 0

for estrazione in data.get("estrazione", []):
    ruota = estrazione["ruotaExtended"].capitalize()
    numeri = estrazione["numeri"]

    for i in range(len(numeri) - 1):
        for j in range(i + 1, len(numeri)):
            n1, n2 = numeri[i], numeri[j]
            update_query = sql.SQL("""
                UPDATE ambos_matrix
                SET ocurrencias = ocurrencias + 1
                WHERE ruota = %s AND numero_1 = %s AND numero_2 = %s
            """)
            cursor.execute(update_query, (ruota, n1, n2))
            update_counter += 1

    if update_counter >= commit_chunk:
        conn.commit()
        print(f"Commit realizado después de {update_counter} actualizaciones.")
        update_counter = 0

if update_counter > 0:
    conn.commit()
    print(f"Commit final realizado con {update_counter} actualizaciones restantes.")

cursor.close()
conn.close()
print("Proceso completado correctamente.")



Introduce la fecha en formato YYYYMMDD: 20250311
Commit final realizado con 110 actualizaciones restantes.
Proceso completado correctamente.
